In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessionpd

/kaggle/input/abuja-comedy-tv/SampleSubmission/SampleSubmission.csv
/kaggle/input/abuja-comedy-tv/test/test.csv
/kaggle/input/abuja-comedy-tv/train/train.csv


In [2]:
df_train = pd.read_csv("/kaggle/input/abuja-comedy-tv/train/train.csv")
df_test = pd.read_csv("/kaggle/input/abuja-comedy-tv/test/test.csv")
sub = pd.read_csv("/kaggle/input/abuja-comedy-tv/SampleSubmission/SampleSubmission.csv")

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
# Check for missing values
missing_values = df_train.isnull().sum()
print("Missing values in each column:\n", missing_values)

In [ ]:
import matplotlib.pyplot as plt

# Plot the distribution of ratings
plt.hist(df_train['Rating'], bins=10, edgecolor='black')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.title('Distribution of Ratings')
plt.show()


In [ ]:
df_train.groupby("Joke_identifier")["Rating"].count().sort_values(ascending=False).head(20)

In [3]:
user_item_matrix = df_train.pivot_table(index='Viewers_ID', columns='Joke_identifier', values='Rating', fill_value=0)

In [ ]:
user_item_matrix.head()

In [ ]:
user_item_matrix.shape

In [3]:
from surprise import KNNWithMeans,KNNBaseline,KNNBasic,KNNWithZScore
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.model_selection import GridSearchCV
import lightgbm as lgb
import os
from surprise import SVD,SVDpp,SlopeOne,NMF,NormalPredictor,BaselineOnly,CoClustering
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

In [4]:
df_train.drop("Response_ID",axis=1,inplace=True)

In [5]:
# Stage 1: Predict Event Quality
event_quality = df_train.groupby('Joke_identifier')['Rating'].mean().reset_index()
event_quality.columns = ['Joke_identifier', 'avg_rating']

# Create user features
user_avg_rating = df_train.groupby('Viewers_ID')['Rating'].mean().reset_index()
user_avg_rating.columns = ['Viewers_ID', 'user_avg_rating']

In [ ]:
df_train.shape

In [ ]:
df_train_filtered.shape

In [6]:
#Reading the dataset
reader = Reader(rating_scale=(-5, 5))
data = Dataset.load_from_df(df_train,reader)

In [ ]:
#Splitting the dataset
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [38]:
svd = SVD(n_factors=500,n_epochs=200,lr_all=0.020,reg_all=0.010)
cross_validate(svd, data, measures=['RMSE', 'MAE'],cv=3)

{'test_rmse': array([2.14957039, 2.15335741, 2.15216043]),
 'test_mae': array([1.70088634, 1.70219311, 1.70353321]),
 'fit_time': (187.28791403770447, 187.74716687202454, 190.00959730148315),
 'test_time': (3.0522565841674805, 2.9203500747680664, 2.5460205078125)}

In [39]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [9]:
sim_options = {
    'name': 'cosine',
    'user_based': False  # item-based
}
knn = KNNBasic(k=100,min_k=5,sim_options=sim_options)
knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
def hybrid_predict(user_id, event_id, svd_weight=0.6):
    # Get SVD prediction
    svd_pred = svd.predict(user_id, event_id).est
    
    # Get KNN prediction
    knn_pred = knn.predict(user_id, event_id).est
    
    # Combine predictions
    hybrid_pred = svd_weight * svd_pred + (1 - svd_weight) * knn_pred
    
    return hybrid_pred

In [ ]:
# Function to predict ratings for a user for events they haven't rated
def predict_new_ratings(user_id):
    # Get all events the user hasn't rated
    user_ratings = df[df['user_id'] == user_id]['event_id']
    all_events = set(df['event_id'])
    events_to_predict = list(all_events - set(user_ratings))
    
    # Get hybrid predictions for all these events
    predictions = [(event_id, hybrid_predict(user_id, event_id))
                   for event_id in events_to_predict]
    
    return predictions

In [ ]:
# Add a placeholder 'Rating' column for compatibility
df_test['Rating'] = 0

In [ ]:
# Convert the test data to a list of tuples
testset = [tuple(x) for x in df_test[['Viewers_ID', 'Joke_identifier','Rating']].values]

In [ ]:
# Make predictions on the test set
test_predictions = [hybrid_predict(uid, iid) for (uid, iid, _) in testset]

In [ ]:
tes = svd.test(testset)

In [ ]:
test_true_ratings = [r for (_, _, r) in testset]

In [ ]:
# Convert predictions to a DataFrame for better readability
pred_df = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in prediction], columns=['Viewers_ID', 'Joke_identifier', 'Rating'])


In [ ]:
# Convert predictions to a DataFrame
predictions_df = pd.DataFrame({
    'Rating': test_predictions
})

# Display the DataFrame
print(predictions_df)


# 3rd Approach

In [ ]:
# Stage 1: Predict Event Quality
event_quality = df_train.groupby('Joke_identifier')['Rating'].mean().reset_index()
event_quality.columns = ['Joke_identifier', 'avg_rating']

# Create user features
user_avg_rating = df_train.groupby('Viewers_ID')['Rating'].mean().reset_index()
user_avg_rating.columns = ['Viewers_ID', 'user_avg_rating']

In [10]:
# Merge features back to the original dataframe
df_with_features = pd.merge(df_train, event_quality, on='Joke_identifier')
df_with_features = pd.merge(df_with_features, user_avg_rating, on='Viewers_ID')


In [33]:
# Train LightGBM model
X = df_with_features[['avg_rating', 'user_avg_rating']]
y = df_with_features['Rating']
lgb_model = lgb.LGBMRegressor(objective='regression', num_leaves=50, learning_rate=0.1, n_estimators=300)
lgb_model.fit(X, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025397 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Number of data points in the train set: 612702, number of used features: 2
[LightGBM] [Info] Start training from score 0.647024


LGBMRegressor(n_estimators=300, num_leaves=50, objective='regression')

In [43]:
# Prepare dictionaries for faster lookups
event_quality_dict = dict(zip(event_quality['Joke_identifier'], event_quality['avg_rating']))
user_avg_rating_dict = dict(zip(user_avg_rating['Viewers_ID'], user_avg_rating['user_avg_rating']))

# Global variables for means
event_quality_mean = event_quality['avg_rating'].mean()
user_avg_rating_mean = user_avg_rating['user_avg_rating'].mean()

def hybrid_predict(user_id, event_id, svd_weight=0.4, knn_weight=0.3, lgb_weight=0.3):
    # SVD prediction
    svd_pred = svd.predict(user_id, event_id).est
    
    # KNN prediction
    knn_pred = knn.predict(user_id, event_id).est
    
    # LightGBM prediction
    event_avg = event_quality_dict.get(event_id, event_quality_mean)
    user_avg = user_avg_rating_dict.get(user_id, user_avg_rating_mean)
    
    lgb_pred = lgb_model.predict([[event_avg, user_avg]])[0]
    
    # Combine predictions
    hybrid_pred = (svd_weight * svd_pred + 
                   knn_weight * knn_pred + 
                   lgb_weight * lgb_pred)
    
    return hybrid_pred

def predict_test_ratings(test_data):
    # Prepare input for batch prediction
    event_avgs = [event_quality_dict.get(eid, event_quality_mean) for eid in test_data['Joke_identifier']]
    user_avgs = [user_avg_rating_dict.get(uid, user_avg_rating_mean) for uid in test_data['Viewers_ID']]
    
    # Batch predict with LightGBM
    lgb_preds = lgb_model.predict(np.column_stack([event_avgs, user_avgs]))
    
    # Batch predict with SVD and KNN
    svd_preds = [svd.predict(uid, eid).est for uid, eid in zip(test_data['Viewers_ID'], test_data['Joke_identifier'])]
    knn_preds = [knn.predict(uid, eid).est for uid, eid in zip(test_data['Viewers_ID'], test_data['Joke_identifier'])]
    
    # Combine predictions
    hybrid_preds = 0.6 * np.array(svd_preds) + 0.2 * np.array(knn_preds) + 0.2 * lgb_preds
    
    # Create results DataFrame
    results = pd.DataFrame({
        'Viewers_ID': test_data['Viewers_ID'],
        'Joke_identifier': test_data['Joke_identifier'],
        'Rating': hybrid_preds
    })
    
    return results

# Make predictions on the test set
test_predictions = predict_test_ratings(df_test)

# Display the first few predictions
print(test_predictions.head())

  Viewers_ID   Joke_identifier    Rating
0         A1  Akpororo Lagos 1  2.257940
1         A1  Akpororo Abuja 1  2.067186
2         A1  Akpororo Abuja 5  1.773886
3         A1  I Go Dye Benin 1  1.802862
4         A1  I Go Dye Benin 3  1.390433


In [41]:
sub["Rating"] = test_predictions["Rating"]

In [42]:
sub.to_csv("Sub20.csv",index=False)

In [ ]:
sub.head()